In [1]:
import pandas as pd

### Westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [2]:
west_count = pd.read_csv("curated_metagenomics/data/westernized_count.csv", index_col=0)

Choosing randomly 100 samples

In [3]:
west_count = west_count.sample(100, axis=1)

In [4]:
west_count

,SchirmerM_2016.metaphlan_bugs_list.stool:G88878,QinN_2014.metaphlan_bugs_list.stool:LD-19,NielsenHB_2014.metaphlan_bugs_list.stool:MH0034,CosteaPI_2017.metaphlan_bugs_list.stool:SID713B025-11-0-0,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Validation_38,VogtmannE_2016.metaphlan_bugs_list.stool:MMRS77018926ST-27-0-0,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_18,KosticAD_2015.metaphlan_bugs_list.stool:PRJNA231909.3115615,SchirmerM_2016.metaphlan_bugs_list.stool:G88980,XieH_2016.metaphlan_bugs_list.stool:YSZC12003_36007,...,LiJ_2017.metaphlan_bugs_list.stool:H3M516118,SchirmerM_2016.metaphlan_bugs_list.stool:G89177,LiJ_2017.metaphlan_bugs_list.stool:H1M513820,VatanenT_2016.metaphlan_bugs_list.stool:G78886,XieH_2016.metaphlan_bugs_list.stool:YSZC12003_36666,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_171,QinJ_2012.metaphlan_bugs_list.stool:T2D-019,LomanNJ_2013.metaphlan_bugs_list.stool:OBK4096,LiSS_2016.metaphlan_bugs_list.stool:FAT_024-22-84-0,NielsenHB_2014.metaphlan_bugs_list.stool:V1_CD41_0
k__Bacteria,26281467,34278107,37394544,64683112,24287257,78034610,31378005,26697032,29489808,92329280,...,50344280,24794748,41826656,10704361,55144692,62582044,39810334,5001488,54867568,117937605
k__Bacteria|p__Bacteroidetes,1973536,23733539,30067832,40031181,10964818,19603690,6782275,13416257,2068739,30976195,...,20484881,11626260,5263111,7189969,38648621,18114663,22085284,1139519,10210089,27001836
k__Bacteria|p__Proteobacteria,43325,2511747,196767,1358371,585546,2254676,379609,816630,98951,335822,...,4309269,73930,1759468,14045,198306,423748,3640922,288942,127334,40012317
k__Bacteria|p__Firmicutes,20531158,7983655,7089164,16616024,12408240,52894965,19616017,8681143,22554049,50385342,...,21682405,10851716,34322954,1275923,12177696,38134358,10692152,3157935,42582960,41578068
k__Bacteria|p__Actinobacteria,3733280,42493,40777,6672943,202481,3280241,3320659,3782999,4706618,4261802,...,3861492,2242842,481124,1632266,2583689,5394989,167402,376573,1469314,9315026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Caudovirales|f__Podoviridae|g__Podoviridae_noname|s__Clostridium_phage_phiCP7R,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Caudovirales|f__Podoviridae|g__Podoviridae_noname|s__Clostridium_phage_phiCP7R|t__PRJNA167663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Staphylococcaceae|g__Staphylococcus|s__Staphylococcus_vitulinus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Non-westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [5]:
non_west_count = pd.read_csv("curated_metagenomics/data/non_westernized_count.csv", index_col=0)

In [6]:
non_west_count = non_west_count.sample(100, axis=1)

### Sediment samples from Archaelogical sediments

In [7]:
sediment_count = pd.read_csv("PRJEB18629/data/sediment_count.csv", index_col=1).drop("Unnamed: 0", 1)

### Marsha's samples

In [8]:
import os

In [9]:
def get_samp_basename(samp_name):
    return("_".join(samp_name.split("_")[0:2]))

In [10]:
def read_sample(samp_name):
    return(pd.read_csv("wibowo_mp/data/"+samp_name, 
                       sep="\t", index_col="#SampleID")
           .rename(columns={'Metaphlan2_Analysis':get_samp_basename(samp_name)}))

In [11]:
wib_metadata = pd.read_csv("wibowo_mp/samples_metadata.txt", sep="\t", index_col=0)

In [12]:
wib_metadata['total_reads'] = wib_metadata["Number of reads (pair 1)"] +  wib_metadata['Number of reads (pair 2)']

In [13]:
soil_samp = ['Lib4_10','Lib4_11','Lib4_12']

In [14]:
wib_samp = [i for i in os.listdir("wibowo_mp/data/")]

In [15]:
wib = read_sample(wib_samp[0])

In [16]:
for i in wib_samp[1:]:
    wib = wib.merge(read_sample(i), left_index=True, right_index=True, how='outer')

In [17]:
wib = wib.fillna(0)

In [18]:
for col in wib.columns:
    wib[col] = wib[col]/100*wib_metadata.loc[col,'total_reads']

### Merging all samples

In [19]:
all_samp = (west_count.merge(non_west_count, left_index=True, right_index=True, how='outer')
            .merge(sediment_count, left_index=True, right_index=True, how='outer')
            .merge(wib, left_index=True, right_index=True, how='outer').fillna(0))

Keeping only Taxons that occur in more than 10% of samples

In [20]:
all_samp = all_samp.loc[all_samp.astype(bool).sum(axis=1) >= int(all_samp.shape[1]/10),:]

Keeping only samples that have 10 or more Taxons

In [21]:
all_samp = all_samp.loc[:,all_samp.astype(bool).sum(axis=0) >= 10]

In [22]:
all_samp

,SchirmerM_2016.metaphlan_bugs_list.stool:G88878,QinN_2014.metaphlan_bugs_list.stool:LD-19,NielsenHB_2014.metaphlan_bugs_list.stool:MH0034,CosteaPI_2017.metaphlan_bugs_list.stool:SID713B025-11-0-0,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Validation_38,VogtmannE_2016.metaphlan_bugs_list.stool:MMRS77018926ST-27-0-0,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_18,KosticAD_2015.metaphlan_bugs_list.stool:PRJNA231909.3115615,SchirmerM_2016.metaphlan_bugs_list.stool:G88980,XieH_2016.metaphlan_bugs_list.stool:YSZC12003_36007,...,Lib4_7,Lib4_4,Lib4_11,Lib4_8,Lib4_5,Lib4_10,Lib4_6,Lib3_9,s_02,Lib4_3
k__Archaea,0.0,0.0,1776.0,0.0,310397.0,39404.0,840573.0,0.0,887444.0,4885.0,...,78761.735712,2.452155e+06,0.0,104488.05607,72376.31679,0.0,1.767970e+06,2.619678e+07,2.577621e+07,817276.595791
k__Archaea|p__Euryarchaeota,0.0,0.0,1776.0,0.0,310397.0,39404.0,840573.0,0.0,887444.0,4885.0,...,78761.735712,2.452155e+06,0.0,104488.05607,72376.31679,0.0,1.767970e+06,2.619678e+07,2.577621e+07,817276.595791
k__Archaea|p__Euryarchaeota|c__Methanobacteria,0.0,0.0,0.0,0.0,310397.0,39404.0,840573.0,0.0,887444.0,4885.0,...,59565.327008,2.452155e+06,0.0,0.00000,72376.31679,0.0,1.767970e+06,2.619678e+07,2.577621e+07,817276.595791
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales,0.0,0.0,0.0,0.0,310397.0,39404.0,840573.0,0.0,887444.0,4885.0,...,59565.327008,2.452155e+06,0.0,0.00000,72376.31679,0.0,1.767970e+06,2.619678e+07,2.577621e+07,817276.595791
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae,0.0,0.0,0.0,0.0,310397.0,39404.0,840573.0,0.0,887444.0,4885.0,...,59565.327008,2.452155e+06,0.0,0.00000,72376.31679,0.0,1.767970e+06,2.619678e+07,2.577621e+07,817276.595791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae,0.0,0.0,0.0,0.0,2623.0,0.0,0.0,0.0,0.0,15025.0,...,0.000000,0.000000e+00,0.0,0.00000,0.00000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus,0.0,0.0,0.0,0.0,2623.0,0.0,0.0,0.0,0.0,15025.0,...,0.000000,0.000000e+00,0.0,0.00000,0.00000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus|s__Dasheen_mosaic_virus,0.0,0.0,0.0,0.0,2623.0,0.0,0.0,0.0,0.0,15025.0,...,0.000000,0.000000e+00,0.0,0.00000,0.00000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus|s__Dasheen_mosaic_virus|t__PRJNA15388,0.0,0.0,0.0,0.0,2623.0,0.0,0.0,0.0,0.0,15025.0,...,0.000000,0.000000e+00,0.0,0.00000,0.00000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


### Preparing metadata 

In [23]:
meta_west = pd.Series(index=west_count.columns, data=['westernized_humans']*west_count.shape[1]).to_frame(name='Env')
meta_west['SourceSink'] = ["source"]*west_count.shape[1]

In [24]:
meta_non_west = pd.Series(index=non_west_count.columns, data=['non_westernized_humans']*non_west_count.shape[1]).to_frame(name='Env')
meta_non_west['SourceSink'] = ["source"]*non_west_count.shape[1]

In [25]:
meta_sediment = pd.Series(index=sediment_count.columns, data=['archeo_sediment']*sediment_count.shape[1]).to_frame(name='Env')
meta_sediment['SourceSink'] = ["source"]*sediment_count.shape[1]

In [26]:
meta_wib_sink = pd.Series(index=wib.columns, data=["-"]*wib.shape[1]).to_frame(name='Env')
meta_wib_sink['SourceSink'] = ['sink']*wib.shape[1]
meta_wib_sink.drop(soil_samp, axis=0, inplace=True)

In [27]:
meta_wib_source = pd.Series(index=soil_samp, data=['archeo_sediment']*len(soil_samp)).to_frame(name="Env")
meta_wib_source['SourceSink'] = ['source']*len(soil_samp)

In [28]:
meta = meta_west.append(meta_non_west).append(meta_wib_sink).append(meta_wib_source)

In [29]:
meta = meta[['SourceSink','Env']]

Checking for concordance between metadata and data

In [30]:
all_samp=all_samp[meta.index.intersection(all_samp.T.index)]

In [31]:
all_samp = all_samp.astype(int)

In [32]:
meta = meta.loc[meta.index.intersection(all_samp.T.index),:]

### Exporting to csv

In [33]:
all_samp.to_csv("all_samp_st2.tsv", sep="\t", index_label='taxon')

In [34]:
meta.to_csv("labels_st2.tsv", sep="\t", index_label='#SampleID')

### Converting counts file to biom format

In [35]:
!biom convert -i all_samp_st2.tsv -o all_samp_st2.biom --table-type="Taxon table" --to-json

Minimum read count for SourceTracker rarefaction depth

In [36]:
all_samp.sum(axis=0).min()

23387215